# Encrypting AES BIE1

### Bitcoin elliptic curve conventions

In [1]:
import binascii
import hashlib
import ecdsa
import codecs

from Crypto.Cipher import AES

import base64
import base58

In [2]:
# secp256k1, http://www.oid-info.com/get/1.3.132.0.10
p_ec = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEFFFFFC2F
r_ec = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEBAAEDCE6AF48A03BBFD25E8CD0364141
b_ec = 0x0000000000000000000000000000000000000000000000000000000000000007
a_ec = 0x0000000000000000000000000000000000000000000000000000000000000000
Gx = 0x79BE667EF9DCBBAC55A06295CE870B07029BFCDB2DCE28D959F2815B16F81798
Gy = 0x483ada7726a3c4655da4fbfc0e1108a8fd17b448a68554199c47d08ffb10d4b8
curve_secp256k1 = ecdsa.ellipticcurve.CurveFp(p_ec, a_ec, b_ec)
generator_secp256k1 = ecdsa.ellipticcurve.Point(curve_secp256k1, Gx, Gy, r_ec)
oid_secp256k1 = (1, 3, 132, 0, 10)
SECP256k1 = ecdsa.curves.Curve("SECP256k1", curve_secp256k1, generator_secp256k1, oid_secp256k1)

In [3]:
def get_point_pubkey(point_input):
    if (point_input.y() % 2) == 1:
        result = '03' + '%064x' % point_input.x()
    else:
        result = '02' + '%064x' % point_input.x()
    return binascii.hexlify(codecs.decode(result, 'hex'))

# Electrum test

### Alice and Bob private and public keys

In [4]:
PA = ecdsa.ellipticcurve.Point(ecdsa.SECP256k1.curve, \
                               81222520315766445915033933156108856478292330375403547871433300150780990928884, \
                               25419995136673019914281218521069623169807792832170161614421324055184750272491, 10)
print('Alice public key =', PA)

CompPA = get_point_pubkey(PA)
print('Alice compressed public key =', CompPA)

Alice public key = (81222520315766445915033933156108856478292330375403547871433300150780990928884,25419995136673019914281218521069623169807792832170161614421324055184750272491)
Alice compressed public key = b'03b392513f4bb63bab6e446c84480fafa5f38be8350e61b80a9c1a098a158937f4'


In [5]:
SB = 99700726144837597339837499243820914618717618365214734329132998527433625976967
print('Bob private key =', SB)

PB = SB * generator_secp256k1
print('Bob public key =', PB)

Bob private key = 99700726144837597339837499243820914618717618365214734329132998527433625976967
Bob public key = (109455533002050838990465911673612665240235545204278096587626452012943280698764,44884149155407838069601222450098554169901247813915580764758522129377407158301)


### Decode Electrum payload

In [6]:
entire_package = 'QklFMQOzklE/S7Y7q25EbIRID6+l84voNQ5huAqcGgmKFYk39EJyEjlPQZ6tcYtBKRikhPe6phPsScTU5NvUUQnZc5HoHg/JRZ0qijLGvueEgMkEor03ruO4cxk0WsxClF13450='

entire_package = binascii.hexlify(base64.b64decode(entire_package))

print(entire_package)

b'4249453103b392513f4bb63bab6e446c84480fafa5f38be8350e61b80a9c1a098a158937f4427212394f419ead718b412918a484f7baa613ec49c4d4e4dbd45109d97391e81e0fc9459d2a8a32c6bee78480c904a2bd37aee3b87319345acc42945d77e39d'


In [7]:
# Split off protocol flag, ephemeral pub key, ciphertext and HMAC

BIE1 = entire_package[:8]
K = entire_package[8:74]             # 8 + 66 = 74  
ciphertext = entire_package[74:]
HMAC = entire_package[-32:]        

print(BIE1, 'in ascii =', binascii.unhexlify(BIE1))
print('K =', K)
print('ciphertext =', ciphertext)
print('HMAC =', HMAC)

b'42494531' in ascii = b'BIE1'
K = b'03b392513f4bb63bab6e446c84480fafa5f38be8350e61b80a9c1a098a158937f4'
ciphertext = b'427212394f419ead718b412918a484f7baa613ec49c4d4e4dbd45109d97391e81e0fc9459d2a8a32c6bee78480c904a2bd37aee3b87319345acc42945d77e39d'
HMAC = b'bd37aee3b87319345acc42945d77e39d'


In [8]:
# Check that K is exactly equal to Alice's public key

CompPA == K

True

### Shared data

In [9]:
SAB = SB * PA
S = get_point_pubkey(SAB)

In [10]:
S

b'02abf18eb5c4c29e54132c95af5192d8c0196c52244f5faf961bd109e9f9a7e332'

In [11]:
IV = hashlib.sha512(S).digest()[0:16]
KAES = hashlib.sha512(S).digest()[16:32]
KHMAC = hashlib.sha512(S).digest()[32:48]

In [12]:
print('IV = ', binascii.hexlify(IV))

IV =  b'fe37fc12cd9fe7130f113128b66ca254'


In [13]:
print(hashlib.sha512(S).digest())
print(IV)
print(KAES)
print(KHMAC)

b'\xfe7\xfc\x12\xcd\x9f\xe7\x13\x0f\x111(\xb6l\xa2T(\xbb\x10\t\xf6m\xf3a.\x1b\x04\x99\x87V]\xe9\xb9"\xa1\x8d/_\xc9\x10z\xc9\x0f\x8c\x9ba7C\xd1f\xd9\xe6Tx\xb3X\xb9-\x96\x97kz\x956'
b'\xfe7\xfc\x12\xcd\x9f\xe7\x13\x0f\x111(\xb6l\xa2T'
b'(\xbb\x10\t\xf6m\xf3a.\x1b\x04\x99\x87V]\xe9'
b'\xb9"\xa1\x8d/_\xc9\x10z\xc9\x0f\x8c\x9ba7C'


### Encrypt

In [14]:
aes = AES.new(KAES, AES.MODE_CBC, IV)

plaintext = b'The answer is no'    
print(plaintext)
print('plaintext =', binascii.hexlify(plaintext))
ciphertext = aes.encrypt(plaintext)
print('ciphertext =', binascii.hexlify(ciphertext))

b'The answer is no'
plaintext = b'54686520616e73776572206973206e6f'
ciphertext = b'0903f7600eac4b06d8ec04bbb40fe239'


This does not match the ciphertext in Electrum